## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-10-14-15-00 +0000,nyt,"Susan Collins Runs for Re-election, in One of ...",https://www.nytimes.com/2026/02/10/us/politics...
1,2026-02-10-14-14-16 +0000,bbc,Seven local authorities set to hike council ta...,https://www.bbc.com/news/articles/c7439dem0wgo...
2,2026-02-10-14-08-02 +0000,startribune,Live: Prior Lake skier Paula Moltzan wins bron...,https://www.startribune.com/2026-winter-olympi...
3,2026-02-10-14-04-00 +0000,wsj,U.S. Retail Sales Unexpectedly Flat in December,https://www.wsj.com/economy/u-s-retail-sales-h...
4,2026-02-10-14-03-00 +0000,wsj,U.S. Natural Gas Futures Steady in Early Trade,https://www.wsj.com/articles/european-gas-pric...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2459/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
124,trump,28
359,new,20
133,epstein,19
130,canada,14
98,year,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
170,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...,91
110,2026-02-10-09-57-46 +0000,nypost,Trump says he will block US-Canada Bridge unle...,https://nypost.com/2026/02/10/us-news/trump-sa...,87
64,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...,87
326,2026-02-09-15-32-32 +0000,nypost,"Ghislaine Maxwell offers to clear Trump, Clint...",https://nypost.com/2026/02/09/us-news/ghislain...,81
204,2026-02-09-23-15-50 +0000,cbc,Trump threatens to block opening of new bridge...,https://www.cbc.ca/news/canada/trump-gordie-ho...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
170,91,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...
58,77,2026-02-10-11-41-00 +0000,wsj,"The U.S. economy looks healthy, but it crossed...",https://www.wsj.com/finance/stocks/healthcare-...
125,71,2026-02-10-08-32-51 +0000,nypost,Nancy Guthrie was supposed to go to pal’s hous...,https://nypost.com/2026/02/10/us-news/nancy-gu...
64,59,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...
192,50,2026-02-10-00-06-00 +0000,wsj,Soaring profits and stocks funnel more of GDP ...,https://www.wsj.com/economy/jobs/capital-labor...
91,42,2026-02-10-10-34-24 +0000,nypost,Bad Bunny wipes Instagram after Super Bowl LX ...,https://nypost.com/2026/02/10/us-news/bad-bunn...
136,40,2026-02-10-06-34-48 +0000,nyt,Air Canada Cancels Flights to Cuba as Cuba Run...,https://www.nytimes.com/2026/02/09/world/ameri...
228,32,2026-02-09-22-08-06 +0000,nyt,How Ilia Malinin and Team U.S.A. Held Off Japa...,https://www.nytimes.com/2026/02/08/world/how-t...
178,32,2026-02-10-01-25-52 +0000,latimes,Federal judge strikes down California mask ban...,https://www.latimes.com/california/story/2026-...
65,31,2026-02-10-11-30-00 +0000,wsj,DuPont de Nemours Posts Flat Sales Amid Weak C...,https://www.wsj.com/business/earnings/dupont-d...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
